In [227]:
from pybaseball import statcast_batter
from pybaseball import batting_stats_bref
from pybaseball import retrosheet
from pybaseball import playerid_lookup
from pybaseball.lahman import *
from pybaseball import playerid_reverse_lookup
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor

In [7]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [171]:
download_lahman()

In [61]:
batting = batting()
pitching = pitching()

In [77]:
recent_bats = batting[batting['yearID'] > 2000]
recent_pitch = pitching[pitching['yearID'] > 2000]
recent_pitch.columns


Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'W', 'L', 'G', 'GS',
       'CG', 'SHO', 'SV', 'IPouts', 'H', 'ER', 'HR', 'BB', 'SO', 'BAOpp',
       'ERA', 'IBB', 'WP', 'HBP', 'BK', 'BFP', 'GF', 'R', 'SH', 'SF', 'GIDP'],
      dtype='object')

In [174]:
career_totals = recent_bats.groupby('playerID').sum()
career_totals.reset_index()

# Limit to only players with 100+ AB
career_totals = career_totals[career_totals['AB'] > 100]

# Set career stat columns
# Batting Average
career_totals['BA'] = career_totals['H'] / career_totals['AB']
# 2B/AB
career_totals['2B/AB'] = career_totals['2B'] / career_totals['AB']
# 3B/AB
career_totals['3B/AB'] = career_totals['3B'] / career_totals['AB']
# HR/AB
career_totals['HR/AB'] = career_totals['HR'] / career_totals['AB']
# RBI/G
career_totals['RBI/G'] = career_totals['RBI'] / career_totals['G']
# R/G
career_totals['R/G'] = career_totals['R'] / career_totals['G']
# BB/G
career_totals['BB/G'] = career_totals['BB'] / career_totals['G']
# SB/G
career_totals['SB/G'] = career_totals['SB'] / career_totals['G']
# HBP/G
career_totals['HBP/G'] = career_totals['HBP'] / career_totals['G']




In [175]:
# Drop columns we don't need
career_totals = career_totals.iloc[:,19:]
career_totals = career_totals.reset_index()

In [176]:
data = pd.read_csv('../meana2020.txt', header=None)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2424 entries, 0 to 2423
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       2424 non-null   object
 1   1       2424 non-null   object
 2   2       2424 non-null   object
 3   3       2424 non-null   object
 4   4       2424 non-null   object
 5   5       798 non-null    object
 6   6       523 non-null    object
 7   7       265 non-null    object
 8   8       2424 non-null   int64 
 9   9       2424 non-null   int64 
 10  10      2424 non-null   int64 
 11  11      2424 non-null   int64 
 12  12      2424 non-null   int64 
 13  13      2424 non-null   object
 14  14      2424 non-null   object
 15  15      2424 non-null   object
dtypes: int64(5), object(11)
memory usage: 303.1+ KB


In [177]:
columns = {0:'game_id', 1:'batter', 2:'batter_hand', 3:'pitcher', 4:'pitcher_hand',
           5:'run_first', 6:'run_second', 7:'run_third', 8:'result', 9:'rbi',
           10:'first_dest', 11:'second_dest', 12:'third_dest',13:'sb_first',
           14:'sb_second', 15:'sb_third'}

new_data = data.rename(columns=columns)

In [178]:

def calc_runs(game_df, batter, hrs):
    runs = hrs
    runs += game_df[(game_df['run_first'] == batter) & (game_df['first_dest'].isin([4,5,6]))].shape[0]
    runs += game_df[(game_df['run_second'] == batter) & (game_df['second_dest'].isin([4,5,6]))].shape[0]
    runs += game_df[(game_df['run_third'] == batter) & (game_df['third_dest'].isin([4,5,6]))].shape[0]
    return runs

def calc_sb(game_df, batter):
    sb = 0 
    sb += game_df[(game_df['run_first'] == batter) & (game_df['sb_first'] == 'T')].shape[0]
    sb += game_df[(game_df['run_second'] == batter) & (game_df['sb_second'] == 'T')].shape[0]
    sb += game_df[(game_df['run_third'] == batter) & (game_df['sb_third'] == 'T')].shape[0]
    return sb

In [179]:
columns = list(career_totals.columns)
columns.extend(['fantasy_ppg', 'starting_pitcher_hand', 'starting_pitcher_era', 'ppg_vs_hand', 'fantasy_points'])


batter_df = pd.DataFrame(columns=columns)
games = new_data['game_id'].unique()
for game in games:
    current_game = new_data[new_data['game_id'] == game]
    
    #date, wind_factor, temp, ballpark_rating = stadium_info(event_file_path, game) 
    
    for batter in current_game['batter'].unique():
        # Convert name to that used by Baseball-Reference
        bbref_name = playerid_reverse_lookup([batter], key_type='retro')
        bbref_name = bbref_name['key_bbref']
        n_index = str(bbref_name).index('\n')
        bbref_name = str(bbref_name)[5:n_index]
        
        
        current_batter = current_game[current_game['batter'] == batter]
        
        starting_pitcher_hand = current_batter.iloc[0, 4]
        singles = current_batter[current_batter['result'] == 20].shape[0]
        doubles = current_batter[current_batter['result'] == 21].shape[0]
        triples = current_batter[current_batter['result'] == 22].shape[0]
        hrs = current_batter[current_batter['result'] == 23].shape[0] 
        rbis = current_batter['rbi'].sum()
        runs = calc_runs(current_game, batter, hrs)
        walks = current_batter[current_batter['result'] == 14].shape[0]
        sb = calc_sb(current_game, batter)
        hbp = current_batter[current_batter['result'] == 16].shape[0]
        games = batter_df[batter_df['playerID'] == bbref_name].shape[0]
        total_fantasy_points = batter_df[batter_df['playerID'] == bbref_name].sum()['fantasy_points']
        if games > 0:
            points_per_game = total_fantasy_points/games
        else:
            points_per_game = 0
        
        total_vs_pitch_hand = batter_df[(batter_df['playerID'] == bbref_name) & (batter_df['starting_pitcher_hand'] == starting_pitcher_hand)].sum()['fantasy_points']
        games_vs_pitch_hand = batter_df[(batter_df['playerID'] == bbref_name) & (batter_df['starting_pitcher_hand'] == starting_pitcher_hand)].shape[0]
        
        
        if games_vs_pitch_hand > 0:
            ppg_vs_hand = total_vs_pitch_hand / games_vs_pitch_hand
        else:
            ppg_vs_hand = points_per_game
        
        
        fantasy_points = 3*singles + 6*doubles + 9*triples + 12*hrs + 3.5*rbis + 3.2*runs + 3*walks + 6*sb + 3*hbp
        
        
        
        
        career_nums = career_totals[career_totals['playerID'] == bbref_name]
        
        career_nums['fantasy_ppg'] = float(points_per_game)
        career_nums['ppg_vs_hand'] = float(ppg_vs_hand)
        career_nums['starting_pitcher_hand'] = starting_pitcher_hand
        career_nums['fantasy_points'] = fantasy_points
        batter_df = batter_df.append(career_nums)
    
       


<ipython-input-179-62293629fa41>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_nums['fantasy_ppg'] = float(points_per_game)
<ipython-input-179-62293629fa41>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_nums['ppg_vs_hand'] = float(ppg_vs_hand)
<ipython-input-179-62293629fa41>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

<ipython-input-179-62293629fa41>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_nums['fantasy_ppg'] = float(points_per_game)
<ipython-input-179-62293629fa41>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_nums['ppg_vs_hand'] = float(ppg_vs_hand)
<ipython-input-179-62293629fa41>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

<ipython-input-179-62293629fa41>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_nums['fantasy_ppg'] = float(points_per_game)
<ipython-input-179-62293629fa41>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_nums['ppg_vs_hand'] = float(ppg_vs_hand)
<ipython-input-179-62293629fa41>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

<ipython-input-179-62293629fa41>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_nums['fantasy_ppg'] = float(points_per_game)
<ipython-input-179-62293629fa41>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_nums['ppg_vs_hand'] = float(ppg_vs_hand)
<ipython-input-179-62293629fa41>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

<ipython-input-179-62293629fa41>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_nums['fantasy_ppg'] = float(points_per_game)
<ipython-input-179-62293629fa41>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_nums['ppg_vs_hand'] = float(ppg_vs_hand)
<ipython-input-179-62293629fa41>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

...

<ipython-input-179-62293629fa41>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_nums['fantasy_ppg'] = float(points_per_game)
<ipython-input-179-62293629fa41>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_nums['ppg_vs_hand'] = float(ppg_vs_hand)
<ipython-input-179-62293629fa41>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

<ipython-input-179-62293629fa41>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_nums['fantasy_ppg'] = float(points_per_game)
<ipython-input-179-62293629fa41>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_nums['ppg_vs_hand'] = float(ppg_vs_hand)
<ipython-input-179-62293629fa41>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

<ipython-input-179-62293629fa41>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_nums['fantasy_points'] = fantasy_points
<ipython-input-179-62293629fa41>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_nums['fantasy_ppg'] = float(points_per_game)
<ipython-input-179-62293629fa41>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

<ipython-input-179-62293629fa41>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_nums['fantasy_ppg'] = float(points_per_game)
<ipython-input-179-62293629fa41>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_nums['ppg_vs_hand'] = float(ppg_vs_hand)
<ipython-input-179-62293629fa41>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

<ipython-input-179-62293629fa41>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_nums['fantasy_ppg'] = float(points_per_game)
<ipython-input-179-62293629fa41>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_nums['ppg_vs_hand'] = float(ppg_vs_hand)
<ipython-input-179-62293629fa41>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [180]:
batter_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 547 entries, 1125 to 1789
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   playerID               547 non-null    object 
 1   BA                     547 non-null    float64
 2   2B/AB                  547 non-null    float64
 3   3B/AB                  547 non-null    float64
 4   HR/AB                  547 non-null    float64
 5   RBI/G                  547 non-null    float64
 6   R/G                    547 non-null    float64
 7   BB/G                   547 non-null    float64
 8   SB/G                   547 non-null    float64
 9   HBP/G                  547 non-null    float64
 10  fantasy_ppg            547 non-null    float64
 11  starting_pitcher_hand  547 non-null    object 
 12  starting_pitcher_era   0 non-null      object 
 13  ppg_vs_hand            547 non-null    float64
 14  fantasy_points         547 non-null    float64
dtypes:

In [155]:
def stadium_info(event_file_path, game_id):
    f = open(event_file_path)
    text = f.read()
    game_idx = text.index(game_id)
    
    date_idx = text.index('date', game_idx)
    date = text[date_idx+5:date_idx+15]
    date = pd.to_datetime(date)
    
    winddir_idx = text.index('winddir,', game_idx)
    winddir = text[winddir_idx+8:winddir_idx+12]
    if winddir in ['ltor', 'rtol', 'unkn']:
        winddir = 0
    elif winddir in ['tocf', 'torf', 'tolf']:
        winddir = 1
    else:
        winddir = -1
    
    windspeed_idx = text.index('windspeed', game_idx)
    windspeed = text[windspeed_idx+10:windspeed_idx+11]
    wind_factor = winddir * int(windspeed)
    
    temp_idx = text.index('info,temp,', game_idx)
    temp = int(text[temp_idx+10:temp_idx+12])
    
    return date, wind_factor, temp, #ballpark_factor
    
stadium_info('../2015ANA.EVA', 'ANA201506290')

FileNotFoundError: [Errno 2] No such file or directory: '../2015ANA.EVA'

In [196]:
batter_df = pd.read_csv('../data/completed_df.csv')

batter_df

,Unnamed: 0,playerID,BA,2B/AB,3B/AB,HR/AB,RBI/G,R/G,BB/G,SB/G,...,position_2.0,position_3.0,position_4.0,position_5.0,position_6.0,position_7.0,position_8.0,position_9.0,position_10.0,position_11.0
0,3083,tabatjo01,0.275126,0.050879,0.009422,0.010678,0.247544,0.408644,0.253438,0.092338,...,0,0,0,0,0,0,0,1,0,0
1,2539,preslal01,0.263383,0.035689,0.013562,0.020700,0.246667,0.348889,0.180000,0.066667,...,0,0,0,0,0,1,0,0,0,0
2,2012,mccutan01,0.284697,0.056807,0.007950,0.040245,0.527520,0.602350,0.522573,0.118120,...,0,0,0,0,0,0,1,0,0,0
3,1174,gordode01,0.285937,0.032233,0.014037,0.004679,0.233533,0.530938,0.167665,0.332335,...,0,0,0,0,1,0,0,0,0,0
4,896,ellisma01,0.262458,0.050225,0.004886,0.020520,0.383275,0.480836,0.305226,0.057143,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5548,1368,hernaen02,0.240089,0.052485,0.006700,0.039643,0.328986,0.365217,0.266667,0.017391,...,0,0,0,0,0,0,0,0,0,1
5549,2610,reddijo01,0.262626,0.048861,0.007752,0.033827,0.442846,0.465228,0.309353,0.048761,...,0,0,0,0,0,0,0,1,0,0
5550,701,culbech01,0.248528,0.047114,0.007067,0.027091,0.259346,0.228972,0.107477,0.028037,...,0,0,0,0,0,0,0,0,0,1
5551,925,ethiean01,0.284792,0.063125,0.007083,0.033750,0.472165,0.440550,0.356701,0.019931,...,0,0,0,0,0,0,0,0,0,1


In [197]:
batter_df = batter_df.dropna()[500:]

In [198]:
# Drop unused categories
batter_df = batter_df.drop(columns=['Unnamed: 0', 'playerID', 'starting_pitcher_hand'])

# Drop pitchers

X = batter_df.drop(columns=['fantasy_points'])
y = batter_df['fantasy_points']

ss = StandardScaler()
ss.fit(X)
X = ss.transform(X)





X_train, X_test, y_train, y_test = train_test_split(X, y)


In [199]:
xg_reg = xgb.XGBRegressor(n_estimators = 1000, learning_rate =.01, max_depth=1)
xg_reg.fit(X_train, y_train)

y_hat_xg = xg_reg.predict(X_test)
mean_squared_error(y_test, y_hat_xg)**.5

7.807264775709884

In [208]:
xg_reg_2 = xgb.XGBRegressor(n_estimators = 1000, learning_rate =.1, max_depth=2)
xg_reg_2.fit(X_train, y_train)
y_hat_xg2 = xg_reg_2.predict(X_test)
mean_squared_error(y_test, y_hat_xg2)**.5

8.024243473692822

In [212]:
xg_reg_3 = xgb.XGBRegressor(n_estimators = 1000, learning_rate =.2, max_depth=1)
xg_reg_3.fit(X_train, y_train)
y_hat_xg3 = xg_reg_3.predict(X_test)
mean_squared_error(y_test, y_hat_xg3)**.5

7.917042937823292

In [201]:
mean_fantasy = y_train.mean()
mean_ = np.full((y_test.shape[0],), mean_fantasy)
mean_squared_error(y_test, mean_)**.5

8.253007919164181

In [202]:
line = LinearRegression()
line.fit(X_train, y_train)
y_hat_line = line.predict(X_test)
mean_squared_error(y_test, y_hat_line)**.5


7.854371070268797

In [275]:
knn = KNeighborsRegressor(n_neighbors=1000)
knn.fit(X_train, y_train)
y_hat_knn = knn.predict(X_test)
mean_squared_error(y_test, y_hat_knn)**.5

7.955130515390784

In [280]:
w1=10
w2=4
w3=5
w4=1
sum_w = w1+w2+w3+w4
ensemble = (y_hat_xg * w1 + y_hat_line * w2 + y_hat_xg2 * w3 + y_hat_knn * w4) / sum_w
mean_squared_error(y_test, ensemble)**.5

7.770369533285392

In [187]:
ensemble

,BA,2B/AB,3B/AB,HR/AB,RBI/G,R/G,BB/G,SB/G,HBP/G,fantasy_ppg,ppg_vs_hand,fantasy_points,wind_factor,temp,starting_p_rpip,pitcher
500,0.252560,0.059727,0.003413,0.013652,0.246637,0.309417,0.197309,0.049327,0.008969,6.150000,8.200000,9.2,0.0,59.0,0.602198,0.0
501,0.291274,0.067689,0.006958,0.033962,0.562062,0.599175,0.608660,0.164948,0.013608,9.340000,9.337500,12.0,0.0,59.0,0.602198,0.0
502,0.284792,0.063125,0.007083,0.033750,0.472165,0.440550,0.356701,0.019931,0.039863,12.500000,11.180000,3.0,0.0,59.0,0.602198,0.0
503,0.256667,0.053111,0.005015,0.015956,0.291262,0.402219,0.258669,0.101942,0.058252,4.314286,3.800000,9.5,0.0,59.0,0.602198,0.0
504,0.220435,0.045412,0.008515,0.038789,0.301587,0.328798,0.260771,0.063492,0.029478,0.000000,0.000000,6.0,0.0,59.0,0.295757,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15521,0.262626,0.048861,0.007752,0.033827,0.442846,0.465228,0.309353,0.048761,0.005596,7.146552,7.086275,0.0,2.0,68.0,0.479537,0.0
15522,0.290158,0.046431,0.018482,0.020436,0.446387,0.581585,0.219697,0.279720,0.027389,9.280000,9.022222,28.2,2.0,68.0,0.479537,0.0
15523,0.277266,0.055044,0.004297,0.015756,0.365924,0.491014,0.340762,0.039540,0.015816,9.753226,8.208333,3.0,2.0,68.0,0.479537,0.0
15524,0.256325,0.060012,0.002746,0.037851,0.489651,0.429495,0.397154,0.016171,0.039457,8.635294,10.780000,3.5,2.0,68.0,0.479537,0.0


1520      3.0
13467     0.0
11154     0.0
8501     22.2
7430      6.0
         ... 
7539      3.0
13416     3.0
10402     3.0
983       0.0
11813     3.0
Name: fantasy_points, Length: 3668, dtype: float64